#### Check, if a metabolite is in a libsbml model

Note: takes way to much time -> maybe still include it in the toolbox, it is already programmed ....

In [ ]:
from refinegems.curation.polish import get_set_of_curies

# @TODO make it a bit more suffisticated
def getAnnotationDict_libsbml(entity):
    try: 
        for cvterm in entity.getCVTerms():
            current_uris = [cvterm.getResourceURI(i) for i in range(cvterm.getNumResources())]
            return get_set_of_curies(current_uris)[0]
    except Exception as e:
        return None
    
def hasAnnotation_libmodel(id, idtype, entitytype, libmodel):
    match entitytype:
        case 'reaction':
            entitylist = libmodel.getListOfReactions()
        case 'species':
            entitylist = libmodel.getListOfSpecies()
        case _:
            mes = f'Unknown entity type: {entitytype}'
            raise ValueError(mes)
        
    found = []
    for r in entitylist:
        annots = getAnnotationDict_libsbml(r)
        if annots and idtype in annots.keys() and id in annots[idtype]:
            found.append(r.getId())
    return found 
